In [1]:
from InvestingWebScrapper import InvestingWebScrapper
from SqlAlquemyInsertHandler import SqlAlquemyInsertHandler

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
sql_handler = SqlAlquemyInsertHandler()
web_scrapper = InvestingWebScrapper()
web_scrapper.open_economic_calendar()

.

In [78]:
df = web_scrapper.read_all_indicators()
print('Read', len(df), 'indicators from', 
      df.iloc[0, 0], 'to', df.iloc[-1, 0])
sql_handler.insert_into_table(df)

c:\Users\lukas\Documents\Master mAI-X\TFM\Data\Investing_EconomicCalendar\InvestingWebScrapper.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, 'Date'] = df_filtered['Imp.'].ffill()


Read 1066 indicators from 2019-12-01 16:30:00 to 2019-12-31 19:00:00
Executing insert sql...
Inserted 200 records
Executing insert sql...
Inserted 200 records
Executing insert sql...
Inserted 200 records
Executing insert sql...
Inserted 200 records
Executing insert sql...
Inserted 200 records
Executing insert sql...
Inserted 66 records


TODO: Continue from 01-01-2020